In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')
if download_last_date == df_adjOHLCV_last_date:
  msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
  raise SystemExit(msg_stop )

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-02-08
df_adjOHLCV_last_date: 2023-02-07


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-02-10   df_Close:  140.73405     Yahoo_Close:  140.73405   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-02-10   df_Close:  375.56805     Yahoo_Close:  375.56802   %_dif_Close: 0.00001
symbol:   SPY   Date: 2022-02-10   df_Close:  442.14691     Yahoo_Close:  442.14691   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
# TODO add list of SPY, VGT, FTSM, QQQ, BNDX 
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'BNDX', 'USO', 'JJC', 'BCI', 'NG']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1314
symbols added:   10
unique symbols:  1318


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[]]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], []]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], [], []]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends
[*********************100%***********************]  118 of 118 completed

symbols_download_err: [[], [], [], []]
downloaded symbols from all chucks
symbols_download_err: [[], [], [], []]


In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

[]

In [9]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [10]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [11]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [12]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [13]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [14]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [15]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [16]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [17]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [18]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15382 entries, 1962-01-02 to 2023-02-08
Columns: 7908 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7883), int64(25)
memory usage: 928.2 MB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15382 entries, 1962-01-02 to 2023-02-08
Columns: 6590 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(6565), int64(25)
memory usage: 773.5 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15382 entries, 1962-01-02 to 2023-02-08
Columns: 1318 entries, A to ZWS
dtypes: float64(1318)
memory usage: 154.8 MB
df_close.info():
None

len(symbols_df):
1318



In [19]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [20]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1318
count_df_close_cols: 1318
count_cols_df_OHLCVA_downloaded: 7908
count_cols_df_adjOHLCV: 6590


In [21]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-02-10   df_Close:  140.73405     Yahoo_Close:  140.73405   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-02-10   df_Close:  375.56805     Yahoo_Close:  375.56802   %_dif_Close: 0.00001
symbol:   SPY   Date: 2022-02-10   df_Close:  442.14691     Yahoo_Close:  442.14691   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [22]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-08,374.290009,380.000000,371.660004,379.320007,373.856598,607500.0
2022-02-09,380.109985,384.670013,379.619995,384.359985,378.823975,702800.0
2022-02-10,382.760010,386.339996,379.929993,382.019989,376.517700,906000.0
2022-02-11,382.989990,401.220001,380.260010,399.320007,393.568573,1772600.0


In [23]:
df_adjOHLCV.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-02-02  156.660004  157.309998  154.770004  155.500000  1725600.0   
2023-02-03  153.169998  155.419998  152.740005  154.550003   992800.0   
2023-02-06  152.729996  154.399994  151.360001  152.240005   771500.0   
2023-02-07  151.490005  154.960007  151.130005  154.509995  1126700.0   
2023-02-08  153.639999  155.139999  153.285995  154.419998   786094.0   

                   AA                                            ...  \
                 Open       High        Low      Close   Volume  ...   
Date                                                             ...   
2023-02-02  54.279999  54.469002  51.919998  53.660000  4665600  ...   
2023-02-03  53.400002  54.380001  52.580002  52.740002  3545900  ...   
2023-02-06  55.139999  55.400002  50.860001  52.020000  5405100  ...   
2023-02-07  52.160000  53.400002  50.865002  53.240002  4359400  ...   
2023-02-08  52.180000  52.679901  50.610001  50.619999  4562803  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-02-02  27.500000  28.969999  27.500000  28.200001  471400.0  22.410000   
2023-02-03  27.809999  28.639999  27.760000  28.080000  298100.0  22.469999   
2023-02-06  27.670000  28.180000  26.490000  26.590000  489300.0  22.209999   
2023-02-07  26.350000  27.639999  25.440001  26.670000  280300.0  22.170000   
2023-02-08  26.290001  26.420000  25.530001  25.850000  345138.0  25.100000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-02-02  22.959999  22.240000  22.719999  1353500.0  
2023-02-03  22.990000  22.430000  22.520000  1329100.0  
2023-02-06  22.370001  21.900000  22.299999  1236700.0  
2023-02-07  22.559999  22.070000  22.370001  2024500.0  
2023-02-08  25.100000  23.565001  24.090000  4406036.0  

[5 rows x 6590 columns]

In [24]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-02-02,440.859985,217.710007,94.000000,457.190002,167.490005,171.789993
2023-02-03,440.640015,216.970001,91.959999,459.079987,169.630005,170.029999
2023-02-06,448.959991,222.009995,91.779999,469.100006,170.250000,171.770004
2023-02-07,452.149994,225.309998,91.220001,468.329987,173.000000,174.179993
2023-02-08,456.059998,225.699997,89.010002,469.649994,170.350006,174.559998


In [25]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-02-02,149.279999,85.480003,23.170000
2023-02-03,147.830002,84.599998,23.049999
2023-02-06,148.350006,82.860001,22.860001
2023-02-07,149.929993,85.089996,23.760000
2023-02-08,148.740005,81.980003,23.440001


In [26]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,BNDX,USO,JJC,BCI,NG
Date,,,,,,,,,,
2023-02-02,416.779999,369.679993,109.330002,311.720001,42.360001,49.090000,66.570000,20.729000,21.320000,6.39
2023-02-03,412.350006,365.760010,108.169998,306.179993,41.689999,48.779999,64.400002,20.570000,20.940001,6.20
2023-02-06,409.829987,360.959991,106.690002,303.589996,41.259998,48.580002,65.500000,20.500000,20.889999,6.16
2023-02-07,415.190002,369.609985,109.279999,309.880005,41.340000,48.459999,67.760002,20.709999,21.180000,6.19
2023-02-08,410.649994,364.929993,107.970001,304.369995,41.259998,48.560001,68.680000,20.559999,21.170000,6.10


In [27]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-02-08,139.889587,385.906616,443.741089
2022-02-09,143.495880,384.444977,450.235718
2022-02-10,140.734055,375.568054,442.146912
2022-02-11,136.303192,386.052765,433.428345
2022-02-14,134.326187,377.068665,432.011322
